<a ><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Logo_INSA_Lyon_%282014%29.svg/langfr-2560px-Logo_INSA_Lyon_%282014%29.svg.png"  width="200" align="left"> </a>
<div style="text-align: right"> <a ><img src="https://2024.debs.org/img/debs-logo_keep.png"  width="200" align="right"> </a> </div>

<br>
<br>
<br>


<h1><center>DEBS 2024 </center></h1>
<h2><center> <span style="font-weight:normal"><font color='#e42618'> 18TH ACM INTERNATIONAL CONFERENCE ON DISTRIBUTED AND EVENT-BASED SYSTEMS (June 25th–28th 2024, Lyon, France)</font>  </span></center></h2>


<h3><center><font color='gray'>ABNER | JONAS | KEVIN | BENJAMIN</font></center></h3>

As a part of the _Cloud Computing and Big Data Application_ course we participate at the Grand Challenge:
    
    
> The 2024 DEBS Grand Challenge focuses on real-time complex event processing of real-world telemetry data provided by Backblaze (https://www.backblaze.com/).The dataset used for the Grand Challenge is based on fine-granular telemetry data about over 200k hard drives in data centers operated by Backblaze. The goal of the challenge is to identify models that exhibit comparable behavior with respect to a set of attributes best suited to predict failures. Further details on the dataset provided, the queries, non-functional requirements and the overall submission process can be found here: https://2024.debs.org/call-for-grand-challenge-solutions/.


This year’s DEBS Grand Challenge requires you to implement two queries
- Query 1: Count of the recent number of failures detected for each vault (group by storage servers) (Continuous Querying)
- Query 2: Use this number to continuously compute a cluster of the drives (Streaming K-Means)

Input data consists of batches and each batch contains
- SMART readings for a list of drives
- ```vault_ids```: a list of vault identifiers of interest for this batch (used in Q1, see below)
- ```cluster_ids```: a list of cluster identifiers of interest for this batch (used in Q2, see below)
- ```day_end```: a flag that marks the end of one day of readings (could not be found - will be created)

#### Requirements
- ```Python 3.9.18``` (conda env)
- ```pip freeze > requirements.txt```
- ```conda env export > environment.yml```

## Our approach

We opted for Apache Spark, not only because some of us already gave a short presentation on Spark, but because of its polyglot nature and vast implementations of different libraries. Also we can easily start a data stream from a folder of ```CSV``` files!

In [73]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
#from pyspark.sql.functions import *
from pyspark.sql.functions import window, sum as _sum

# Exploration

In [74]:
spark = SparkSession.builder.appName("StreamingFromCSV").getOrCreate()

# load df from csv
df = spark.read.csv("streaming_data_dir/drivedata-small.csv", header=True, inferSchema=True)

In [75]:
# show only rows where failure is not 0
df.where(df.failure != 0).show()


+-------------------+-------------+--------------------+-------+--------+------------------+-------------------------+---------------+-------------------+---------------------------+------------------+------------------------+-----------------+--------------------+---------------------+------------------------+--------------------------------+-------------------------+----------------------------------+--------------------+--------------------+----------------------------+-----------------------+----------------------------+----------------------------+------------------------+---------------------------+----------------------------+---------------------------+--------------------------+-------------------------+--------------------------+---------------+-----------------+---------------------+-----------------+----------------------+-----------------------+--------------------+
|               date|serial_number|               model|failure|vault_id|s1_read_error_rate|s2_throughput_pe

In [76]:
# show the column names and types
df.printSchema()


root
 |-- date: timestamp (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- failure: integer (nullable = true)
 |-- vault_id: integer (nullable = true)
 |-- s1_read_error_rate: integer (nullable = true)
 |-- s2_throughput_performance: integer (nullable = true)
 |-- s3_spin_up_time: integer (nullable = true)
 |-- s4_start_stop_count: integer (nullable = true)
 |-- s5_reallocated_sector_count: integer (nullable = true)
 |-- s7_seek_error_rate: long (nullable = true)
 |-- s8_seek_time_performance: integer (nullable = true)
 |-- s9_power_on_hours: integer (nullable = true)
 |-- s10_spin_retry_count: integer (nullable = true)
 |-- s12_power_cycle_count: integer (nullable = true)
 |-- s173_wear_leveling_count: long (nullable = true)
 |-- s174_unexpected_power_loss_count: integer (nullable = true)
 |-- s183_sata_downshift_count: integer (nullable = true)
 |-- s187_reported_uncorrectable_errors: integer (nullable = true)
 |-- s188_command_

In [77]:
# load cluster centers from csv
cluster_centers = spark.read.csv("cluster_data_dir/clusters.csv", header=True, inferSchema=True)

In [78]:
# show the column names and types
cluster_centers.printSchema()

root
 |-- cluster: integer (nullable = true)
 |-- s1_read_error_rate: double (nullable = true)
 |-- s2_throughput_performance: double (nullable = true)
 |-- s3_spin_up_time: double (nullable = true)
 |-- s4_start_stop_count: double (nullable = true)
 |-- s5_reallocated_sector_count: double (nullable = true)
 |-- s7_seek_error_rate: double (nullable = true)
 |-- s8_seek_time_performance: double (nullable = true)
 |-- s9_power_on_hours: double (nullable = true)
 |-- s10_spin_retry_count: double (nullable = true)
 |-- s12_power_cycle_count: double (nullable = true)
 |-- s173_wear_leveling_count: double (nullable = true)
 |-- s174_unexpected_power_loss_count: double (nullable = true)
 |-- s183_sata_downshift_count: double (nullable = true)
 |-- s187_reported_uncorrectable_errors: double (nullable = true)
 |-- s188_command_timeout: double (nullable = true)
 |-- s189_high_fly_writes: double (nullable = true)
 |-- s190_airflow_temperature_cel: double (nullable = true)
 |-- s191_g_sense_error_

In [79]:
# drop _c35: string (nullable = true) from cluster_centers
cluster_centers = cluster_centers.drop("_c35")

_c35: string (nullable = true) is just an empty cell

We use date and vault ID to cluster all features s1 through s242 and re-write the centers

In [80]:

schema = StructType([
    StructField("date", TimestampType(), True),
    StructField("serial_number", StringType(), True),
    StructField("model", StringType(), True),
    StructField("failure", IntegerType(), True),
    StructField("vault_id", IntegerType(), True),
    StructField("s1_read_error_rate", IntegerType(), True),
    StructField("s2_throughput_performance", IntegerType(), True),
    StructField("s3_spin_up_time", IntegerType(), True),
    StructField("s4_start_stop_count", IntegerType(), True),
    StructField("s5_reallocated_sector_count", IntegerType(), True),
    StructField("s7_seek_error_rate", IntegerType(), True),
    StructField("s8_seek_time_performance", IntegerType(), True),
    StructField("s9_power_on_hours", IntegerType(), True),
    StructField("s10_spin_retry_count", IntegerType(), True),
    StructField("s12_power_cycle_count", IntegerType(), True),
    StructField("s173_wear_leveling_count", IntegerType(), True),
    StructField("s174_unexpected_power_loss_count", IntegerType(), True),
    StructField("s183_sata_downshift_count", IntegerType(), True),
    StructField("s187_reported_uncorrectable_errors", IntegerType(), True),
    StructField("s188_command_timeout", IntegerType(), True),
    StructField("s189_high_fly_writes", IntegerType(), True),
    StructField("s190_airflow_temperature_cel", IntegerType(), True),
    StructField("s191_g_sense_error_rate", IntegerType(), True),
    StructField("s192_power_off_retract_count", IntegerType(), True),
    StructField("s193_load_unload_cycle_count", IntegerType(), True),
    StructField("s194_temperature_celsius", IntegerType(), True),
    StructField("s195_hardware_ecc_recovered", IntegerType(), True),
    StructField("s196_reallocated_event_count", IntegerType(), True),
    StructField("s197_current_pending_sector", IntegerType(), True),
    StructField("s198_offline_uncorrectable", IntegerType(), True),
    StructField("s199_udma_crc_error_count", IntegerType(), True),
    StructField("s200_multi_zone_error_rate", IntegerType(), True),
    StructField("s220_disk_shift", IntegerType(), True),
    StructField("s222_loaded_hours", IntegerType(), True),
    StructField("s223_load_retry_count", IntegerType(), True),
    StructField("s226_load_in_time", IntegerType(), True),
    StructField("s240_head_flying_hours", IntegerType(), True),
    StructField("s241_total_lbas_written", IntegerType(), True),
    StructField("s242_total_lbas_read", IntegerType(), True)
])

#read from csv file
#drivedata = spark.readStream.csv("streaming_data_dir/drivedata-small.csv", schema=schema, header=True)

# Read the CSV files as a data stream
streamingData = spark.readStream.schema(schema).option("maxFilesPerTrigger", 1).csv("streaming_data_dir")
# print true if both are true
print((streamingData.schema == schema)==( streamingData.isStreaming == True))

True


# First Query

- Size = 30 days
- Slide = 1 day
- For every vault $v$, count the number of failures $NF_v$ in a sliding window $W$
- For a given day $i$, $NF_v^i$ is the count of failures in the window that starts at $i-31$ (included) and ends at $i-1$ (included)
- The first window closes at day 0: you can assume 0 failures for every day $i <= 0$
- Each batch of input data will contain the identifiers of 5 vaults: you are to return the current value of $NF_v$ for those vaults

In [15]:

# Apply a window function to the streaming data
windowedData = streamingData \
    .withWatermark("date", "31 days") \
    .groupBy(
        streamingData.vault_id,
        window(streamingData.date, "30 days", "1 day"),
        streamingData.model
    ) \
    .agg(_sum("failure").alias("total_failures"))

# Select only the date and vault_id fields
selectedData = windowedData.select("window.start", "vault_id", "total_failures")
filteredData = selectedData.filter(selectedData.total_failures > 0)

# Write the windowed data stream out to a memory sink for testing
query = filteredData \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("filteredData") \
    .option("numRows", 50) \
    .option("truncate", "false") \
    .start()



24/01/26 16:06:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/temporary-8e16d53e-195d-44ee-86c4-c9b438f75afc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/01/26 16:06:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
query.awaitTermination()

In [69]:
query.stop()

## Second query

- For each reading at day $i$ regarding a drive $d$ belonging to vault $v$, add $NF_v^i$ to the reading
- Rescale the smart values within the provided ranges
- Compute dynamic K-means clustering:
    - Assign incoming readings to the nearest centroid
    - At the end of one day, which is marked by a ```day_end``` flag in the batch, update the centroids positions with the average coordinates of all the readings currently associated to that centroid
    - The initial coordinates of centroids are provided 
    - Each batch of input data will contain a list of cluster identifiers ($0 - 49$) 
    - Return the number of drives associated to each of those clusters

In [82]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import StreamingKMeans

# Load initial center points from a CSV file
centers_df = spark.read.csv("cluster_data_dir/clusters.csv", header=True, inferSchema=True)

# drop _c35: string (nullable = true) from cluster_centers
centers_df = centers_df.drop("_c35")

# Convert the DataFrame to a list of DenseVector
centers = [Vectors.dense(row) for row in centers_df.collect()]
# Define the model
model = StreamingKMeans(k=len(centers), decayFactor=1.0)


In [84]:
# create list of weights for each center point
weights = [1.0 for center in centers]


model.setInitialCenters(centers, weights)


In [89]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the streaming data
streamingData = spark.readStream.schema(schema).csv("streaming_data_dir")# Define a query
query = streamingData.writeStream.outputMode("append").format("console").start()

# Wait for the query to start
query.awaitTermination()

# Now you can convert the streaming DataFrame to an RDD
rdd = streamingData.rdd
# Create a SparkContext
sc = SparkContext.getOrCreate()

# Create a StreamingContext with a batch interval of 1 second
ssc = StreamingContext(sc, 1)

# Assuming that your streaming data is in the form of DataFrame named streamingData
# Convert DataFrame to RDD
rdd = streamingData.rdd

# Convert RDD to DStream
queue = [rdd]
stream = ssc.queueStream(queue)

# Now you can train the model on the DStream
model.trainOn(stream)

24/01/26 17:35:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/temporary-78b93846-4b73-4690-865d-0185b053138d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/01/26 17:35:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+--------------+--------------------+-------+--------+------------------+-------------------------+---------------+-------------------+---------------------------+------------------+------------------------+-----------------+--------------------+---------------------+------------------------+--------------------------------+-------------------------+----------------------------------+--------------------+--------------------+----------------------------+-----------------------+----------------------------+----------------------------+------------------------+---------------------------+----------------------------+---------------------------+--------------------------+-------------------------+--------------------------+---------------+-----------------+---------------------+-----------------+----------------------+-----------------------+--------------------+
|         

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/jonas/Documents/Uni/Erasmus/Cloud Computing for Big Data/Cloud-Computing-and-Big-Data-Applications/.conda/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/jonas/Documents/Uni/Erasmus/Cloud Computing for Big Data/Cloud-Computing-and-Big-Data-Applications/.conda/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/jonas/Documents/Uni/Erasmus/Cloud Computing for Big Data/Cloud-Computing-and-Big-Data-Applications/.conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 